*Ce notebook couvre la phase de modélisation, de sélection et d'évaluation des modèles pour la prédiction de la longévité des joueurs en NBA.*

Il comprend :

- *Gestion des classes déséquilibrées*
-  *L'entraînement et l'évaluation de plusieurs modèles (Random Forest, Balanced Random Forest, XGBoost, SVC)*
- *L'optimisation des hyperparamètres avec GridSearchCV et Optuna*

#### 1. Importation des Bibliothèques

Nous utilisons les bibliothèques suivantes pour la modélisation er l'évaluation des modèles.

In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV , StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
from sklearn.ensemble import RandomForestClassifier 
import optuna
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.metrics import confusion_matrix, recall_score, precision_score, f1_score, balanced_accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Users/ibtissama/anaconda3/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/ibtissama/anaconda3/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/Users/ibtissama/anaconda3/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/Users/ibtissama/anaconda3/lib/python3.10/site-packages/traitlets/config/application.py", line 992, in launch_instance
    app.st

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Users/ibtissama/anaconda3/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/ibtissama/anaconda3/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/Users/ibtissama/anaconda3/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/Users/ibtissama/anaconda3/lib/python3.10/site-packages/traitlets/config/application.py", line 992, in launch_instance
    app.st

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Users/ibtissama/anaconda3/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/ibtissama/anaconda3/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/Users/ibtissama/anaconda3/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/Users/ibtissama/anaconda3/lib/python3.10/site-packages/traitlets/config/application.py", line 992, in launch_instance
    app.st

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Users/ibtissama/anaconda3/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/ibtissama/anaconda3/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/Users/ibtissama/anaconda3/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/Users/ibtissama/anaconda3/lib/python3.10/site-packages/traitlets/config/application.py", line 992, in launch_instance
    app.st

AttributeError: _ARRAY_API not found

Dans le cadre de l'évaluation des modèles de classification, plusieurs versions de la fonction de scoring ont été développées pour affiner l'analyse des performances :

*Version 1 - score_classifier :*

Utilise une validation croisée K-Fold simple / stratified.
Calcule la matrice de confusion ainsi que les métriques pour les deux classes séparément (Classe 0 et Classe 1).
Initialement, la version de base ne prenait en compte que le recall global, ce qui était insuffisant dans notre contexte.
Désormais, cette fonction analyse séparément la classe majoritaire (1) et la classe minoritaire (0), ce qui est essentiel pour éviter les erreurs coûteuses.

*Version 2 - score_classifier_with_tuning :*

Ajoute une optimisation des hyperparamètres à l'aide de GridSearchCV.
Permet d'identifier les meilleurs paramètres pour chaque modèle afin d'améliorer sa performance.


*Version 3 - score_classifier_with_optuna :*

Utilise Optuna, une approche avancée d'optimisation des hyperparamètres, qui est plus efficace et rapide que GridSearchCV.
Recherche automatiquement les meilleurs hyperparamètres en fonction de la performance du modèle.
Permet d'améliorer encore davantage la classification en évitant un exploration exhaustive du paramètre space.


👉 *Pour plus de détails sur les fonctions de scoring, veuillez vous référer au rapport technique.*

Importation des fonctions de scoring

In [3]:
from scoring_kfold import score_classifier
from scoring_optim import score_classifier_with_tuning
from scoring_optuna import score_classifier_with_optuna

#### 2. Chargement et Préparation des Données

Nous importons les données prétraitées dans le fichier précédent pour les utiliser dans la phase de modélisation et d'évaluation.

In [4]:
# Chargement des données
df = pd.read_csv("NBA_transf.csv")
df = df.drop(columns=["Unnamed: 0"])  # Suppression de la colonne inutile

# Extraction des labels et des features
names = df['Name'].values.tolist()  # Noms des joueurs
labels = df['TARGET_5Yrs'].values  # Labels (Classe 0 = <5 ans, Classe 1 = ≥5 ans)
paramset = df.drop(['TARGET_5Yrs', 'Name'], axis=1).columns.values  # Noms des features
df_vals = df.drop(['TARGET_5Yrs', 'Name'], axis=1).values  # Valeurs des features

print(f"Taille du dataset: {df_vals.shape}")


Taille du dataset: (1328, 19)


Ce dataset contient l'ensemble des statistiques des joueurs, prétraitées dans l'étape précédente.

#### 3. Équilibrage des Classes avec SMOTE

Dans notre jeu de données, la classe 1 (joueurs ayant une carrière ≥5 ans) est sur-représentée par rapport à la classe 0 (joueurs ayant une carrière <5 ans).

Un modèle entraîné sur ces données non équilibrées risque de privilégier la classe majoritaire et de sous-estimer les joueurs ayant une carrière courte.

Nous avons testé deux stratégies d’équilibrage :

Pondération des classes dans les modèles (via class_weight='balanced').
Sur-échantillonnage de la classe minoritaire avec SMOTE (Synthetic Minority Over-sampling Technique).

📌 Après comparaison des performances des modèles avec les deux techniques, SMOTE a donné de meilleurs.
SMOTE génère artificiellement des échantillons similaires à la classe minoritaire, ce qui permet au modèle d’avoir plus d’exemples pour apprendre à bien prédire les joueurs à carrière courte.



👉 *Pour plus de détails sur l’analyse comparative des techniques d’équilibrage, voir la section dédiée dans le document technique.*

In [5]:
# Application de SMOTE pour équilibrer les classes
oversample = SMOTE()
over_X, over_y = oversample.fit_resample(df_vals, labels)

print(f"Taille du dataset après SMOTE: {over_X.shape}")

Taille du dataset après SMOTE: (1648, 19)


SMOTE permet d'obtenir une meilleure balance entre précision et recall pour la classe minoritaire, tout en évitant de simplement ajuster les poids dans les modèles.

#### 4. Choix des Modèles de Classification

Pour cette tâche de classification, nous avons sélectionné trois modèles adaptés aux problématiques d’apprentissage supervisé sur des données déséquilibrées :

1️⃣ Random Forest (RF) -> Modèle basé sur un ensemble d’arbres de décision.

2️⃣ Balanced Random Forest (BRF) -> Variante du Random Forest, où chaque arbre est entraîné sur un échantillon équilibré de la classe minoritaire et majoritaire.

3️⃣ XGBoost (XGB) -> Algorithme de boosting optimisé pour la classification binaire.

👉 *Pour plus de détails sur le choix des modèles, voir la section dédiée dans le document technique.*

#### 5. Évaluation des Modèles & Comparaison

Nous appliquons différentes stratégies d’évaluation sur chaque modèle pour comparer leurs performances et identifier le meilleur.

Trois méthodes d’évaluation sont utilisées :

1️⃣ score_classifier → Validation croisée KFold simple / stratified.

2️⃣ score_classifier_with_tuning → Optimisation des hyperparamètres avec GridSearchCV.

3️⃣ score_classifier_with_optuna → Optimisation avancée avec Optuna.

💡 Objectif : Nous allons tester chaque modèle avec ces trois stratégies et analyser leur précision, recall et F1-score.

##### 5.1 Random Forest (RF)

In [7]:
# Version 1 - kfold simple
rf = RandomForestClassifier(random_state=42)
score_classifier(over_X, rf, over_y,"kfold")


📊 Confusion Matrix:
[[634. 190.]
 [256. 568.]]

🎯 Performance Metrics:
🔹 Precision: Class 1 (Long Career Players): 0.7511
🔹 Recall: Class 1 (Long Career Players): 0.6894
🔹 F1-score: Class 1 (Long Career Players): 0.7176
🔹 Precision: Class 0 (Short Career Players): 0.7135
🔹 Recall: Class 0 (Short Career Players): 0.7699
🔹 F1-score: Class 0 (Short Career Players): 0.7395

📌 Full Classification Report (Averaged Over 5 Folds):
              precision    recall  f1-score   support

         0.0       0.71      0.77      0.74       824
         1.0       0.75      0.69      0.72       824

    accuracy                           0.73      1648
   macro avg       0.73      0.73      0.73      1648
weighted avg       0.73      0.73      0.73      1648



Dans cette première version, le modèle a été testé avec des hyperparamètres par défaut, sans optimisation spécifique. 

On observe les résultats suivants :

✅ F1-score Classe 1 (Longue carrière) : 0.7176

✅ F1-score Classe 0 (Courte carrière) : 0.7395

✅ Accuracy globale : 0.73

🔍 Analyse :

Le modèle montre une bonne capacité de généralisation, avec une précision et un rappel équilibrés entre les deux classes.
Toutefois, il ne parvient pas encore à parfaitement différencier les joueurs ayant une longue carrière, avec un recall de 0.6894 pour la classe 1.
Légère sous-estimation des joueurs ayant une longue carrière, ce qui signifie que certains joueurs prometteurs ne sont pas bien identifiés.

In [8]:
#Version 2 - Optimisation GridSearchCV
param_grid_brf = {
    "n_estimators": [100, 300, 500],
    "max_depth": [3, 5, 7, None],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "max_features": ['sqrt', 'log2', None],
}

rf = RandomForestClassifier(random_state=42)

score_classifier_with_tuning(over_X, rf, over_y, param_grid_brf,5,"kfold")


Fitting 5 folds for each of 324 candidates, totalling 1620 fits



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Users/ibtissama/anaconda3/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/ibtissama/anaconda3/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/Users/ibtissama/anaconda3/lib/python3.10/site-packages/joblib/externals/loky/backend/popen_loky_posix.py", line 180, in <module>
    exitcode = process_obj._bootstrap()
  File "/Users/ibtissama/anaconda3/lib/python3.10/multiprocessing/process.py", line 314, in _boots

Fitting 5 folds for each of 324 candidates, totalling 1620 fits
Fitting 5 folds for each of 324 candidates, totalling 1620 fits
Fitting 5 folds for each of 324 candidates, totalling 1620 fits
Fitting 5 folds for each of 324 candidates, totalling 1620 fits

📊 Average Confusion Matrix:
[[629. 195.]
 [239. 585.]]

🎯 Performance Metrics:
🔹 Precision (Class 1 - Long Career Players): 0.7507
🔹 Recall (Class 1 - Majority): 0.7100
🔹 F1-score (Class 1 - Long Career Players): 0.7291
🔹 Precision (Class 0 - Short Career Players): 0.7250
🔹 Recall (Class 0 - Minority): 0.7630
🔹 F1-score (Class 0 - Short Career Players): 0.7429

📜 Full Classification Report (average over folds):
              precision    recall  f1-score   support

         0.0       0.72      0.76      0.74       824
         1.0       0.75      0.71      0.73       824

    accuracy                           0.74      1648
   macro avg       0.74      0.74      0.74      1648
weighted avg       0.74      0.74      0.74      1648



L’ajout d’une optimisation par recherche exhaustive (GridSearchCV) a permis de trouver les meilleurs hyperparamètres du modèle. 

Cette approche a entraîné une amélioration des performances :

✅ F1-score Classe 1 (Longue carrière) : 0.7291 (+1.15%)

✅ F1-score Classe 0 (Courte carrière) : 0.7429 (+0.34%)

✅ Accuracy globale : 0.74 (+1 point)

🔍 Analyse :

Le modèle fait de meilleurs choix, notamment pour la classe 1, avec une légère augmentation du recall (0.7100 contre 0.6894 précédemment).
Le F1-score de la classe 0 (joueurs avec une carrière courte) s’améliore également, ce qui signifie que le modèle identifie mieux les joueurs qui ne dépasseront pas 5 ans en NBA.
Amélioration globale de l’équilibre entre précision et rappel, grâce à un réglage plus fin des paramètres du modèle.

In [9]:
#Vesion 3 - Optimisation avec Optuna
rf = RandomForestClassifier(random_state=42)
rf_model = score_classifier_with_optuna(over_X, rf, over_y, 50, cv=5, use_stratified=False)
rf_model

[I 2025-03-06 15:38:49,925] A new study created in memory with name: no-name-9b13b0df-45e1-4204-bb11-9f4ec9b10681


[I 2025-03-06 15:39:00,533] Trial 0 finished with value: 0.7232866644209365 and parameters: {'n_estimators': 800, 'max_depth': 10, 'min_samples_split': 7, 'min_samples_leaf': 4, 'max_features': 'sqrt'}. Best is trial 0 with value: 0.7232866644209365.
[I 2025-03-06 15:39:03,537] Trial 1 finished with value: 0.7274719457895569 and parameters: {'n_estimators': 200, 'max_depth': None, 'min_samples_split': 6, 'min_samples_leaf': 2, 'max_features': 'log2'}. Best is trial 1 with value: 0.7274719457895569.
[I 2025-03-06 15:39:07,201] Trial 2 finished with value: 0.7183056136532509 and parameters: {'n_estimators': 300, 'max_depth': 10, 'min_samples_split': 3, 'min_samples_leaf': 9, 'max_features': 'log2'}. Best is trial 1 with value: 0.7274719457895569.
[I 2025-03-06 15:39:09,678] Trial 3 finished with value: 0.7223443102386662 and parameters: {'n_estimators': 200, 'max_depth': None, 'min_samples_split': 19, 'min_samples_leaf': 5, 'max_features': 'log2'}. Best is trial 1 with value: 0.727471945


✅ Best Hyperparameters Found: {'n_estimators': 100, 'max_depth': None, 'min_samples_split': 13, 'min_samples_leaf': 2, 'max_features': None}

📊 Average Confusion Matrix:
[[630. 194.]
 [237. 587.]]

🎯 Performance Metrics:
🔹 Precision (Class 1 - Long Career Players): 0.7523
🔹 Recall (Class 1 - Majority): 0.7123
🔹 F1-score (Class 1 - Long Career Players): 0.7310
🔹 Precision (Class 0 - Short Career Players): 0.7272
🔹 Recall (Class 0 - Minority): 0.7644
🔹 F1-score (Class 0 - Short Career Players): 0.7447

📜 Full Classification Report:
              precision    recall  f1-score   support

         0.0       0.73      0.76      0.75       824
         1.0       0.75      0.71      0.73       824

    accuracy                           0.74      1648
   macro avg       0.74      0.74      0.74      1648
weighted avg       0.74      0.74      0.74      1648



RandomForestClassifier(max_features=None, min_samples_leaf=2,
                       min_samples_split=13, random_state=42)

L’utilisation d’Optuna, une technique plus avancée et adaptative d’optimisation des hyperparamètres, a permis d’obtenir les meilleurs résultats observés jusque-là :

✅ F1-score Classe 1 (Longue carrière) : 0.7310 (+1.34% par rapport à la V1)

✅ F1-score Classe 0 (Courte carrière) : 0.7447 (+0.52% par rapport à la V1)

✅ Accuracy globale : 74 

🔍 Analyse :

L’optimisation dynamique via Optuna a permis de mieux ajuster les hyperparamètres, ce qui se traduit par une meilleure séparation des classes.
Le recall pour la classe 1 (0.7123) continue d’augmenter, ce qui signifie que plus de joueurs prometteurs sont correctement identifiés.
Le modèle réduit encore davantage les erreurs sur la classe 0, atteignant un recall de 0.7644 (meilleure valeur observée).
Meilleure balance globale entre recall et précision, ce qui signifie que le modèle est plus fiable pour éviter les faux positifs et les faux négatifs.

👉 *Le modèle final (Version 3 - Optuna) est plus performant que les versions précédentes, offrant une meilleure capacité de prédiction pour identifier les joueurs ayant une longue carrière NBA.*

##### 5.2 Balanced Random Forest (BRF)

In [10]:
# Version 1 - KFold Stratifié
brf = BalancedRandomForestClassifier(random_state=42)

score_classifier(df_vals, brf, labels,"stratified")



📊 Confusion Matrix:
[[324. 180.]
 [239. 585.]]

🎯 Performance Metrics:
🔹 Precision: Class 1 (Long Career Players): 0.7654
🔹 Recall: Class 1 (Long Career Players): 0.7099
🔹 F1-score: Class 1 (Long Career Players): 0.7359
🔹 Precision: Class 0 (Short Career Players): 0.5768
🔹 Recall: Class 0 (Short Career Players): 0.6428
🔹 F1-score: Class 0 (Short Career Players): 0.6069

📌 Full Classification Report (Averaged Over 5 Folds):
              precision    recall  f1-score   support

         0.0       0.58      0.64      0.61       504
         1.0       0.76      0.71      0.74       824

    accuracy                           0.68      1328
   macro avg       0.67      0.68      0.67      1328
weighted avg       0.69      0.68      0.69      1328



Dans cette première approche, nous avons utilisé les hyperparamètres par défaut du modèle sans optimisation spécifique.

✅ F1-score Classe 1 (Longue carrière) : 0.7359

✅ F1-score Classe 0 (Courte carrière) : 0.6069

✅ Accuracy globale : 0.68

🔍 Analyse :

Le modèle favorise fortement la classe majoritaire (Classe 1 - joueurs avec une longue carrière), avec un recall de 0.7099. Cela signifie qu'il parvient à capturer un bon nombre de joueurs qui auront une longue carrière.
Cependant, le recall de la classe 0 (0.6428) est relativement faible, indiquant que le modèle ne détecte pas bien les joueurs ayant une carrière courte.
Le déséquilibre des classes semble impacter les performances globales, avec une précision faible pour la classe 0 (0.5768)

In [11]:
# Version 2 - Optimisation GridSearchCV
param_grid_brf = {
    "n_estimators": [100, 300, 500],
    "max_depth": [3, 5, 7, None],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "max_features": ['sqrt', 'log2', None],
}

brf = BalancedRandomForestClassifier(random_state=42)

score_classifier_with_tuning(over_X, brf, over_y, param_grid_brf,5,"stratified")


Fitting 5 folds for each of 324 candidates, totalling 1620 fits



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Users/ibtissama/anaconda3/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/ibtissama/anaconda3/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/Users/ibtissama/anaconda3/lib/python3.10/site-packages/joblib/externals/loky/backend/popen_loky_posix.py", line 180, in <module>
    exitcode = process_obj._bootstrap()
  File "/Users/ibtissama/anaconda3/lib/python3.10/multiprocessing/process.py", line 314, in _boots

Fitting 5 folds for each of 324 candidates, totalling 1620 fits
Fitting 5 folds for each of 324 candidates, totalling 1620 fits
Fitting 5 folds for each of 324 candidates, totalling 1620 fits
Fitting 5 folds for each of 324 candidates, totalling 1620 fits

📊 Average Confusion Matrix:
[[615. 209.]
 [254. 570.]]

🎯 Performance Metrics:
🔹 Precision (Class 1 - Long Career Players): 0.7309
🔹 Recall (Class 1 - Majority): 0.6918
🔹 F1-score (Class 1 - Long Career Players): 0.7102
🔹 Precision (Class 0 - Short Career Players): 0.7095
🔹 Recall (Class 0 - Minority): 0.7464
🔹 F1-score (Class 0 - Short Career Players): 0.7270

📜 Full Classification Report (average over folds):
              precision    recall  f1-score   support

         0.0       0.71      0.75      0.73       824
         1.0       0.73      0.69      0.71       824

    accuracy                           0.72      1648
   macro avg       0.72      0.72      0.72      1648
weighted avg       0.72      0.72      0.72      1648



Nous avons ensuite appliqué une optimisation avec GridSearchCV, permettant de tester plusieurs combinaisons d’hyperparamètres et d’améliorer la capacité du modèle à différencier les deux classes.

✅ F1-score Classe 1 (Longue carrière) :0.7102  

✅ F1-score Classe 0 (Courte carrière) : 0.7270

✅ Accuracy globale : 0.72(+4 points)

🔍 Analyse :

Amélioration notable du recall de la classe 0 (0.7464 vs 0.6428 en V1) → Le modèle identifie mieux les joueurs ayant une courte carrière, ce qui réduit les faux positifs (éviter de classer un joueur à faible potentiel comme une future star).

Équilibre entre précision et recall → La précision pour les deux classes augmente, passant de 0.5768 à 0.7095 pour la classe 0, ce qui montre une meilleure capacité du modèle à ne pas confondre les joueurs des deux catégories.
Le recall de la classe 1 baisse légèrement (0.6918 vs 0.7099 en V1), mais reste stable et permet toujours d’identifier correctement les talents prometteurs.

In [12]:
# Version 3 - Optimisation avec Optuna
brf = BalancedRandomForestClassifier(random_state=42)
brf_model = score_classifier_with_optuna(over_X, brf, over_y, 50, cv=5, use_stratified=True)
brf_model

[I 2025-03-06 16:40:58,769] A new study created in memory with name: no-name-279a25d1-0f80-4752-9a56-ef78d8492549


[I 2025-03-06 16:41:02,954] Trial 0 finished with value: 0.7168198721680967 and parameters: {'n_estimators': 200, 'max_depth': 10, 'min_samples_split': 19, 'min_samples_leaf': 1, 'max_features': 'log2'}. Best is trial 0 with value: 0.7168198721680967.
[I 2025-03-06 16:41:47,574] Trial 1 finished with value: 0.7125850349150689 and parameters: {'n_estimators': 900, 'max_depth': None, 'min_samples_split': 5, 'min_samples_leaf': 8, 'max_features': None}. Best is trial 0 with value: 0.7168198721680967.
[I 2025-03-06 16:42:18,589] Trial 2 finished with value: 0.6891094670636922 and parameters: {'n_estimators': 900, 'max_depth': 5, 'min_samples_split': 19, 'min_samples_leaf': 5, 'max_features': None}. Best is trial 0 with value: 0.7168198721680967.
[I 2025-03-06 16:42:25,627] Trial 3 finished with value: 0.7165092354322209 and parameters: {'n_estimators': 400, 'max_depth': None, 'min_samples_split': 5, 'min_samples_leaf': 10, 'max_features': 'sqrt'}. Best is trial 0 with value: 0.716819872168


✅ Best Hyperparameters Found: {'n_estimators': 700, 'max_depth': 15, 'min_samples_split': 6, 'min_samples_leaf': 7, 'max_features': 'log2'}

📊 Average Confusion Matrix:
[[617. 207.]
 [242. 582.]]

🎯 Performance Metrics:
🔹 Precision (Class 1 - Long Career Players): 0.7369
🔹 Recall (Class 1 - Majority): 0.7063
🔹 F1-score (Class 1 - Long Career Players): 0.7209
🔹 Precision (Class 0 - Short Career Players): 0.7198
🔹 Recall (Class 0 - Minority): 0.7488
🔹 F1-score (Class 0 - Short Career Players): 0.7336

📜 Full Classification Report:
              precision    recall  f1-score   support

         0.0       0.72      0.75      0.73       824
         1.0       0.74      0.71      0.72       824

    accuracy                           0.73      1648
   macro avg       0.73      0.73      0.73      1648
weighted avg       0.73      0.73      0.73      1648



BalancedRandomForestClassifier(max_depth=15, max_features='log2',
                               min_samples_leaf=7, min_samples_split=6,
                               n_estimators=700, random_state=42)

L’optimisation via Optuna permet une exploration plus fine et adaptative des hyperparamètres, améliorant la classification globale.

✅ F1-score Classe 1 (Longue carrière) : 0.7209 

✅ F1-score Classe 0 (Courte carrière) : 0.7336 

✅ Accuracy globale : 0.73  (+5 points)

🔍 Analyse :

Hausse du recall pour la classe 1 (0.7063 vs 0.6918 en V2) → Le modèle détecte encore mieux les joueurs qui auront une carrière longue.
Précision plus stable pour la classe 0 (0.7198) → Moins de faux positifs, ce qui signifie que le modèle réduit les erreurs d’investissement sur des joueurs qui ne performeront pas.
Recall de la classe 0 atteint  0.7488 → Le modèle identifie encore mieux les joueurs avec une carrière courte, renforçant la prise de décision des investisseurs NBA.
Équilibre parfait entre recall et précision, ce qui permet d’avoir un modèle robuste et fiable.

👉 *Le modèle final (Version 3 - Optuna) est plus performant que les versions précédentes, offrant une meilleure capacité de prédiction pour identifier les joueurs ayant une longue carrière NBA.*

#####  5.3 XGBoost (XGB)

In [13]:
# Version 1 - KFold Stratifié

# Initialize XGBoost classifier with class weighting
xgb_clf = xgb.XGBClassifier(
    objective='binary:logistic',
    random_state=42,
    eval_metric="logloss",
    tree_method="hist"
)

# Call the scoring function with the updated XGBoost classifier
score_classifier(over_X, xgb_clf, over_y,"stratified")



📊 Confusion Matrix:
[[604. 220.]
 [238. 586.]]

🎯 Performance Metrics:
🔹 Precision: Class 1 (Long Career Players): 0.7279
🔹 Recall: Class 1 (Long Career Players): 0.7112
🔹 F1-score: Class 1 (Long Career Players): 0.7186
🔹 Precision: Class 0 (Short Career Players): 0.7183
🔹 Recall: Class 0 (Short Career Players): 0.7329
🔹 F1-score: Class 0 (Short Career Players): 0.7248

📌 Full Classification Report (Averaged Over 5 Folds):
              precision    recall  f1-score   support

         0.0       0.72      0.73      0.73       824
         1.0       0.73      0.71      0.72       824

    accuracy                           0.72      1648
   macro avg       0.72      0.72      0.72      1648
weighted avg       0.72      0.72      0.72      1648



✅ F1-score Classe 1 (Longue carrière) : 0.7186

✅ F1-score Classe 0 (Courte carrière) : 0.7248

✅ Accuracy globale : 0.72

🔍 Analyse :

Le modèle montre une bonne précision générale et une balance entre les classes.
La précision de la classe 1 (0.7279) est correcte.
La classe 0 est également bien prise en compte avec un recall de 0.7329.

In [14]:
# Version 2 - Optimisation GridSearchCV


xgb_clf = xgb.XGBClassifier(
    objective='binary:logistic',  
    random_state=42,
    eval_metric="logloss",  
    use_label_encoder=False,
    tree_method="hist"  
)


# Define parameter grid for tuning
param_grid_xgb = {
    'n_estimators': [100, 300, 1000],  # Number of trees
    'max_depth': [3, 5, 7],  # Tree depth
    'learning_rate': [0.01, 0.1, 0.2],  # Step size shrinkage
    'subsample': [0.7, 0.9],  # Fraction of samples used per tree
    'colsample_bytree': [0.7, 0.9],  # Fraction of features used per tree
}


score_classifier_with_tuning(over_X, xgb_clf, over_y, param_grid_xgb,5,"stratified")


Fitting 5 folds for each of 108 candidates, totalling 540 fits



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Users/ibtissama/anaconda3/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/ibtissama/anaconda3/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/Users/ibtissama/anaconda3/lib/python3.10/site-packages/joblib/externals/loky/backend/popen_loky_posix.py", line 180, in <module>
    exitcode = process_obj._bootstrap()
  File "/Users/ibtissama/anaconda3/lib/python3.10/multiprocessing/process.py", line 314, in _boots

Fitting 5 folds for each of 108 candidates, totalling 540 fits


/Users/ibtissama/anaconda3/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [17:19:42] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/ibtissama/anaconda3/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [17:19:42] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/ibtissama/anaconda3/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [17:19:42] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/ibtissama/anaconda3/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [17:19:42] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserW

Fitting 5 folds for each of 108 candidates, totalling 540 fits


...

/Users/ibtissama/anaconda3/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [17:21:04] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/ibtissama/anaconda3/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [17:21:04] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/ibtissama/anaconda3/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [17:21:04] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/ibtissama/anaconda3/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [17:21:04] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserW

Fitting 5 folds for each of 108 candidates, totalling 540 fits


/Users/ibtissama/anaconda3/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [17:21:44] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/ibtissama/anaconda3/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [17:21:44] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/ibtissama/anaconda3/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [17:21:44] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/ibtissama/anaconda3/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [17:21:44] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserW


📊 Average Confusion Matrix:
[[599. 225.]
 [241. 583.]]

🎯 Performance Metrics:
🔹 Precision (Class 1 - Long Career Players): 0.7202
🔹 Recall (Class 1 - Majority): 0.7076
🔹 F1-score (Class 1 - Long Career Players): 0.7133
🔹 Precision (Class 0 - Short Career Players): 0.7155
🔹 Recall (Class 0 - Minority): 0.7269
🔹 F1-score (Class 0 - Short Career Players): 0.7207

📜 Full Classification Report (average over folds):
              precision    recall  f1-score   support

         0.0       0.71      0.73      0.72       824
         1.0       0.72      0.71      0.71       824

    accuracy                           0.72      1648
   macro avg       0.72      0.72      0.72      1648
weighted avg       0.72      0.72      0.72      1648



/Users/ibtissama/anaconda3/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [17:22:24] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


✅ F1-score Classe 1 (Longue carrière) : 0.7133

✅ F1-score Classe 0 (Courte carrière) : 0.7207 

✅ Accuracy globale : 0.72

🔍 Analyse :

Dans ce cas l'optimisation n'a pa permit d'avoir des performances mieux que la version précédente.

In [15]:
# Version 3 - Optimisation avec Optuna


xgb_clf = xgb.XGBClassifier(
    objective='binary:logistic',  
    random_state=42,
    eval_metric="logloss",  
    use_label_encoder=False,
    tree_method="hist"  
)

XGB_model = score_classifier_with_optuna(over_X, xgb_clf, over_y, 50, cv=5, use_stratified=True)
XGB_model

[I 2025-03-06 17:26:58,208] A new study created in memory with name: no-name-b78f889e-a24a-4e64-8ac9-a9deeb624421


/Users/ibtissama/anaconda3/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [17:26:58] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/ibtissama/anaconda3/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [17:26:58] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/ibtissama/anaconda3/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [17:26:59] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/ibtissama/anaconda3/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [17:26:59] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserW


✅ Best Hyperparameters Found: {'n_estimators': 400, 'max_depth': 7, 'learning_rate': 0.0062749366799965565, 'subsample': 0.975738982375914, 'colsample_bytree': 0.5493036130608416}


/Users/ibtissama/anaconda3/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [17:28:41] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/ibtissama/anaconda3/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [17:28:41] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/ibtissama/anaconda3/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [17:28:42] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/ibtissama/anaconda3/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [17:28:42] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserW


📊 Average Confusion Matrix:
[[624. 200.]
 [235. 589.]]

🎯 Performance Metrics:
🔹 Precision (Class 1 - Long Career Players): 0.7455
🔹 Recall (Class 1 - Majority): 0.7148
🔹 F1-score (Class 1 - Long Career Players): 0.7294
🔹 Precision (Class 0 - Short Career Players): 0.7282
🔹 Recall (Class 0 - Minority): 0.7573
🔹 F1-score (Class 0 - Short Career Players): 0.7421

📜 Full Classification Report:
              precision    recall  f1-score   support

         0.0       0.73      0.76      0.74       824
         1.0       0.75      0.71      0.73       824

    accuracy                           0.74      1648
   macro avg       0.74      0.74      0.74      1648
weighted avg       0.74      0.74      0.74      1648



XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.5493036130608416, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='logloss', feature_types=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.0062749366799965565,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=7, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=400, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

✅ F1-score Classe 1 (Longue carrière) : 0.7294

✅ F1-score Classe 0 (Courte carrière) : 0.7421 

✅ Accuracy globale :  0.74 (+2 points)

🔍 Analyse :

Meilleure performance globale avec une précision et un rappel plus équilibrés.
La classe 1 est mieux identifiée, avec un recall de 0.7148, réduisant ainsi les erreurs où un joueur prometteur est mal classé.
Le recall de la classe 0 atteint 0.7573, ce qui signifie que le modèle est aussi plus efficace pour détecter les joueurs qui échoueront.
La précision des deux classes s’améliore, indiquant un modèle plus robuste et plus fiable pour l’aide à la décision.


👉 *Optuna a permis d’obtenir les meilleures performances, avec un équilibre optimal entre recall et précision pour les deux classes.*

#### 6. Analyse des Résultats et Sélection du Meilleur Modèle

L’objectif est de choisir le modèle de classification offrant le meilleur compromis entre précision et recall, tout en assurant une bonne généralisation sur l’ensemble des joueurs.

1- Random Forest (RF)

Meilleure précision (0.7523) pour la classe 1 (joueurs avec longue carrière). Meilleur F1-score global (0.7310 pour classe 1 et 0.7447 pour classe 0). Bon rappel pour la classe 0 (0.7644), garantissant que les joueurs qui échoueront en NBA sont bien identifiés. 

🔻 Léger déficit en rappel pour la classe 1 (0.7123), ce qui signifie qu’il rate quelques talents prometteurs.

2️- Balanced Random Forest (BalancedRF)

Bonne gestion du déséquilibre des classes, mais précision et rappel plus faibles que RF. F1-score global plus faible (0.7209 et 0.7336), ce qui montre une performance légèrement inférieure à RF et XGBoost.

🔻 Moins performant que RF et XGBoost pour la détection des talents prometteurs.

3️- XGBoost

Très bon équilibre entre précision et rappel pour les deux classes. F1-score légèrement inférieur à RF (0.7294 pour la classe 1 et 0.7421 pour la classe 0).Meilleur rappel (0.7148) pour la classe 1, ce qui signifie qu'il identifie plus de joueurs prometteurs que RF.

🔻 Précision légèrement inférieure à RF (0.7455 vs. 0.7523) → peut générer plus de faux positifs (mauvais investissements).


*Le meilleur modèle est le Random Forest (RF) optimisé avec Optuna.*

Meilleur compromis entre précision et rappel pour la classe 1 (joueurs avec longue carrière).Meilleur F1-score global, garantissant un bon équilibre entre les erreurs de classification.Meilleure précision générale, réduisant ainsi les faux positifs (évite de sélectionner des joueurs qui ne performeront pas).


In [16]:
# Enregistrer le meilleut modèle
import joblib
joblib.dump(rf_model, "rf_model.pkl")
print("✅ Modèle RandomForest enregistré avec succès sous 'rf_model.pkl'")

✅ Modèle RandomForest enregistré avec succès sous 'rf_model.pkl'
